In [50]:
# Import the modules
import pandas as pd
import hvplot.pandas
from pathlib import Path
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [51]:
# Read in the CSV file as a Pandas DataFrame
sales_df = pd.read_csv(
    Path("Resources/US_Regional_Sales_Data.csv"),
    index_col="OrderNumber", 
    parse_dates=True, 
    infer_datetime_format=True
)

# Review the DataFrame
sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_15116\1859464474.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  sales_df = pd.read_csv(
C:\Users\lasot\AppData\Local\Temp\ipykernel_15116\1859464474.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df = pd.read_csv(


,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,31/12/17,31/5/18,14/6/18,19/6/18,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,31/12/17,31/5/18,22/6/18,2/7/2018,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,31/12/17,31/5/18,21/6/18,1/7/2018,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,31/12/17,31/5/18,2/6/2018,7/6/2018,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,10/4/2018,31/5/18,16/6/18,26/6/18,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


DATA CLEANING 

In [52]:
#show data type for each column
sales_df.dtypes

Sales Channel        object
WarehouseCode        object
ProcuredDate         object
OrderDate            object
ShipDate             object
DeliveryDate         object
CurrencyCode         object
_SalesTeamID          int64
_CustomerID           int64
_StoreID              int64
_ProductID            int64
Order Quantity        int64
Discount Applied    float64
Unit Cost            object
Unit Price           object
dtype: object

In [53]:
#change columns ProcuredDate, OrderDate, ShipDate, and DeliveryDate from a format day/month/year to month/day/year

sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], dayfirst=True, errors='coerce')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], dayfirst=True, errors='coerce')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], dayfirst=True, errors='coerce')


sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], format='%d/%m/%Y')
sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], format='%d/%m/%Y')
sales_df['ShipDate'] = pd.to_datetime(sales_df['ShipDate'], format='%d/%m/%Y')
sales_df['DeliveryDate'] = pd.to_datetime(sales_df['DeliveryDate'], format='%d/%m/%Y')



sales_df.head()

C:\Users\lasot\AppData\Local\Temp\ipykernel_15116\1789869082.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['ProcuredDate'] = pd.to_datetime(sales_df['ProcuredDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_15116\1789869082.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['OrderDate'] = pd.to_datetime(sales_df['OrderDate'], dayfirst=True, errors='coerce')
C:\Users\lasot\AppData\Local\Temp\ipykernel_15116\1789869082.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  sales_df['ShipDate'] = pd.to_datetime(sales_df[

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,"$1,001.18","$1,963.10"
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,"$3,348.66","$3,939.60"
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,$781.22,"$1,775.50"
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,"$1,464.69","$2,324.90"
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,"$1,476.14","$1,822.40"


In [54]:
# Remove the dollar sign, commas, and extra spaces, then convert to float for 'Unit Cost'
sales_df['Unit Cost'] = sales_df['Unit Cost'].str.replace('[$,]', '', regex=True).str.strip().astype(float)

# Remove the dollar sign, commas, and extra spaces, then convert to float for 'Unit Price'
sales_df['Unit Price'] = sales_df['Unit Price'].str.replace('[$,]', '', regex=True).str.strip().astype(float)


sales_df.head()


,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price
OrderNumber,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4


In [55]:
#add column "delivery_time" to the dataframe which is the difference between the order date and the delivery date
sales_df["delivery_time_days"] = sales_df["DeliveryDate"] - sales_df["OrderDate"]
#remove "days" from the column and convert to integer
sales_df["delivery_time_days"] = sales_df["delivery_time_days"].astype(str).str.replace('days', '').astype(int)
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days
OrderNumber,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26


In [56]:
#create a new column "price_before_disc" which is the price of the item before the discount is applied
sales_df["price_before_disc"] = sales_df["Unit Price"] / (1 - sales_df["Discount Applied"])
sales_df.head()

,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,_StoreID,_ProductID,Order Quantity,Discount Applied,Unit Cost,Unit Price,delivery_time_days,price_before_disc
OrderNumber,,,,,,,,,,,,,,,,,
SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,259,12,5,0.08,1001.18,1963.1,19,2133.804348
SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,196,27,3,0.08,3348.66,3939.6,32,4282.173913
SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,213,16,1,0.05,781.22,1775.5,31,1868.947368
SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,107,23,8,0.08,1464.69,2324.9,7,2527.065217
SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,111,26,8,0.10,1476.14,1822.4,26,2024.888889


IMPACT OF VARIABLES ON DELIVERY TIMES and delivery time prediction - based on sales channel, sales team, and warehouse 